## adapter.bin 파일로 불러와서 확인해보기
- 이건 이제 안해봐도 될 듯 합니다.궁금한 사람만..!

In [1]:
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from utils.prompter import Prompter
test_prompte_name = "multi"
prompter = Prompter(f"{test_prompte_name}")

start = time.time()
peft_model_id = "./model/lora_finetuned/new_2/"
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, 
                                             quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model.eval()
model.config.use_cache = True

/opt/conda/envs/final/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/envs/final/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /opt/conda/envs/final/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/opt/conda/envs/final/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /opt/conda/envs/final did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/opt/conda/envs/final/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/opt/conda/envs/final/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/opt/conda/envs/final/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/usr/local/cuda/lib64/libcud

In [2]:
def gen(x):
    inputs = tokenizer(
        x, 
        return_tensors='pt', 
        return_token_type_ids=False
    )
    inputs = {name: tensor.to(model.device) for name, tensor in inputs.items()} # move to same device as model
    gened = model.generate(
        **inputs,
        min_new_tokens=30,
        max_new_tokens=128,
        temperature=0.5,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
        # no_repeat_ngram_size=2,
        # repetition_penalty=1.05,
        # top_k=20,
        # top_p=0.95,
        # typical_p=0.9
    )
    
    gened_list = gened[0].tolist()
    
    try:
        eos_token_position = gened_list.index(2)
        gened_list = gened_list[:eos_token_position]
    except ValueError:
        pass
    
    print(tokenizer.decode(gened_list))

messages = [
            # '오늘 날씨가 맑아서 기분이 좋아요 !', 
            # '내일 시험인데 너무 떨려서 잠이 안와요. 어떡하죠?',
            '나는 커서 연기를 하는 배우가 될거야! 그게 나의 꿈이야',
            '이제 일 그만하고 집에가서 쉬고싶어, 넘 힘들어 ㅠㅠ',
            # '친구하고 싸웠는데 어떻게 화해를 하면 좋을지 모르겠어'
            ]
for message in messages:
    prompt = prompter.generate_prompt(message)

    # model.generate()
    print('model.generate() inference 방법')
    start = time.time()

    gen(prompt)

    end = time.time()
    print(f'걸린 시간 : {int(end-start)}초')

    print('##################################################################')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


model.generate() inference 방법


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]


[현재 대화]
input: 나는 커서 연기를 하는 배우가 될거야! 그게 나의 꿈이야

response: 연기하는 배우가 되고 싶은 마음은 알겠어! 너의 꿈을 이루기 위해서는 너의 꿈을 더 자세히 말해줘! 내가 더 좋은 방법을 알려줄께![추가 질문] 달한[추가 질문](1): 너는(2): 1. 너는! 너의 꿈을 이루기 위해 무엇을 하고 있니! (3.나는 너를 사랑하기 위해 더 열심히 노력하고 있어! 너의 꿈을 이루기 위해 너의 하루를 더 보여줘!너의 꿈이 더 궁금해! 너의 꿈을 이루기
걸린 시간 : 15초
##################################################################
model.generate() inference 방법
이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]


[현재 대화]
input: 이제 일 그만하고 집에가서 쉬고싶어, 넘 힘들어 ㅠㅠ

response: 힘들고 피곤하고 지친 련(")(DOM) ANDROIUDELIV("DOM")DOMINE(ALLOWEEKLY)ANDRE(ALLOWEEKLIVE)DOMINE(ANDRELLALIVE)DOMINE(ANDRELLALLOWEEIVEDOMINE(ANDRELLANDIVE(S340-DOMINE(ANDRE
걸린 시간 : 14초
##################################################################


## inference 대화하면서 확인해보기
- prompt template은 본인이 실험할 때 학습시킨 걸로 바꿔주세요(multi 그냥 쓰면 그대로!)
- .evn 파일에 허깅페이스와 openai token이 필요합니다. 만약 openai billing이 등록되어있지 않거나 크레딧이 없으면 오류가 뜹니다.

In [1]:
import torch
import time
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
import torch
from dotenv import load_dotenv

load_dotenv()

test_model_name = "new_2"
MODEL = f"../finetuning/model/lora_merged/{test_model_name}"

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model.eval()
model.config.use_cache = True

/opt/conda/envs/final/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:17<00:00,  5.81s/it]


In [2]:
pipe = pipeline(
    'text-generation',
    model = model,
    tokenizer = tokenizer,
    device=0,
    min_new_tokens=30,
    max_new_tokens=256,
    early_stopping=True,
    do_sample=True,
    eos_token_id=2,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    temperature=0.7,
    top_k=20,
    top_p=0.95,
)
local_llm = HuggingFacePipeline(pipeline=pipe)

## Vector store-backed memory
[Vector store-backed memory link](https://python.langchain.com/docs/modules/memory/how_to/vectorstore_retriever_memory)

VectorStoreRetrieverMemory는 VectorDB에 메모리를 저장하고 호출될 때마다 가장 '두드러진' 상위 K개의 문서를 쿼리합니다.

이는 상호작용의 순서를 명시적으로 추적하지 않는다는 점에서 다른 대부분의 메모리 클래스와 다릅니다.

이 경우 "문서"는 이전 대화 스니펫입니다. 이는 대화 초기에 AI가 들었던 관련 정보를 참조하는 데 유용할 수 있습니다.

### chain에서 활용

In [5]:
import json
import faiss
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS

embedding_size = 1536 # Dimensions of the OpenAIEmbeddings
index = faiss.IndexFlatL2(embedding_size)
embedding_fn = OpenAIEmbeddings().embed_query
vectorstore = FAISS(embedding_fn, index, InMemoryDocstore({}), {})
retriever = vectorstore.as_retriever(search_kwargs=dict(k=2))
memory = VectorStoreRetrieverMemory(retriever=retriever)

template = """이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
{history}

[현재 대화]
### 명령어: {### 명령어}

### 응답:
"""

PROMPT = PromptTemplate(
    input_variables=["history", "### 명령어"], template=template
)

conversation_with_vectors = ConversationChain(
    llm=local_llm,
    prompt=PROMPT,
    memory=memory,
    verbose=True,
    input_key='### 명령어',
    output_key='### 응답'
)

# 대답 생성 함수
def get_response(question):
    input_dict = {'### 명령어': question}
    res = conversation_with_vectors.predict(**input_dict)
    return res

In [6]:
res = get_response('요즘 하는 프로젝트 때문에 스트레스를 많이 받고 있어')
print(res)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new  chain...
Prompt after formatting:
이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]


[현재 대화]
### 명령어: 요즘 하는 프로젝트 때문에 스트레스를 많이 받고 있어

### 응답:


> Finished chain.
프로젝트가 제 능력 이상으로 어렵거나 불확실하게 느껴져 걱정되시겠어요, 친구! 하지만 그럴 때일수록 마음챙김이 중요합니다... 이렇게 생각해 보면 어떨까요? 이 경험은 나 자신만 겪었던 것도 아니며 세상 모든 사람들이 같지 않다는 사실입니다.. 그렇죠?"


In [7]:
res = get_response('그렇지..! 다들 하는건데 나만 못할 순 없지! 좋아 잘 해볼게')
print(res)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new  chain...
Prompt after formatting:
이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
### 명령어: 요즘 하는 프로젝트 때문에 스트레스를 많이 받고 있어
### 응답: 프로젝트가 제 능력 이상으로 어렵거나 불확실하게 느껴져 걱정되시겠어요, 친구! 하지만 그럴 때일수록 마음챙김이 중요합니다... 이렇게 생각해 보면 어떨까요? 이 경험은 나 자신만 겪었던 것도 아니며 세상 모든 사람들이 같지 않다는 사실입니다.. 그렇죠?"

[현재 대화]
### 명령어: 그렇지..! 다들 하는건데 나만 못할 순 없지! 좋아 잘 해볼게

### 응답:


> Finished chain.
좋아요, 시작합시다!! 저한테 말할 땐 좀 내려놓으세요." [웃음 포인트로 마무리하면 좋습니다 ^-^)/~♥


In [8]:
res = get_response('갑자기 무슨소리야 ㅋㅋㅋ 나 이제 밥먹으러 갈꺼야')
print(res)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new  chain...
Prompt after formatting:
이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
### 명령어: 그렇지..! 다들 하는건데 나만 못할 순 없지! 좋아 잘 해볼게
### 응답: 좋아요, 시작합시다!! 저한테 말할 땐 좀 내려놓으세요." [웃음 포인트로 마무리하면 좋습니다 ^-^)/~♥
### 명령어: 요즘 하는 프로젝트 때문에 스트레스를 많이 받고 있어
### 응답: 프로젝트가 제 능력 이상으로 어렵거나 불확실하게 느껴져 걱정되시겠어요, 친구! 하지만 그럴 때일수록 마음챙김이 중요합니다... 이렇게 생각해 보면 어떨까요? 이 경험은 나 자신만 겪었던 것도 아니며 세상 모든 사람들이 같지 않다는 사실입니다.. 그렇죠?"

[현재 대화]
### 명령어: 갑자기 무슨소리야 ㅋㅋㅋ 나 이제 밥먹으러 갈꺼야

### 응답:


> Finished chain.
좋아, 난 여기서 기다릴 테니 맛있지만 건강식이 되도록 할게요!" (밥 먹으면서 할 수있다; 다음 단계 진행 ㄱㄱ


In [9]:
res = get_response('너가 건강식이 된다고..? 그게 무슨소리니... 암튼 안녕')
print(res)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new  chain...
Prompt after formatting:
이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
### 명령어: 갑자기 무슨소리야 ㅋㅋㅋ 나 이제 밥먹으러 갈꺼야
### 응답: 좋아, 난 여기서 기다릴 테니 맛있지만 건강식이 되도록 할게요!" (밥 먹으면서 할 수있다; 다음 단계 진행 ㄱㄱ
### 명령어: 요즘 하는 프로젝트 때문에 스트레스를 많이 받고 있어
### 응답: 프로젝트가 제 능력 이상으로 어렵거나 불확실하게 느껴져 걱정되시겠어요, 친구! 하지만 그럴 때일수록 마음챙김이 중요합니다... 이렇게 생각해 보면 어떨까요? 이 경험은 나 자신만 겪었던 것도 아니며 세상 모든 사람들이 같지 않다는 사실입니다.. 그렇죠?"

[현재 대화]
### 명령어: 너가 건강식이 된다고..? 그게 무슨소리니... 암튼 안녕

### 응답:


> Finished chain.
죄송해요 방금 한 말 못 들으셨나요.... 저 오늘 아침부터 저녁식사 전까질 수도 없습니다!! 그 동안 최선 다할게요~^-/-->


In [10]:
res = get_response('잠실가서 딤섬먹을꺼야')
print(res)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new  chain...
Prompt after formatting:
이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
### 명령어: 갑자기 무슨소리야 ㅋㅋㅋ 나 이제 밥먹으러 갈꺼야
### 응답: 좋아, 난 여기서 기다릴 테니 맛있지만 건강식이 되도록 할게요!" (밥 먹으면서 할 수있다; 다음 단계 진행 ㄱㄱ
### 명령어: 너가 건강식이 된다고..? 그게 무슨소리니... 암튼 안녕
### 응답: 죄송해요 방금 한 말 못 들으셨나요.... 저 오늘 아침부터 저녁식사 전까질 수도 없습니다!! 그 동안 최선 다할게요~^-/-->

[현재 대화]
### 명령어: 잠실가서 딤섬먹을꺼야

### 응답:


> Finished chain.
네! 잠심 가셔서 꼭 드셔보셨으면 좋겠어요~! 잘 다녀오시고 나중에도 얘기 나눠요 :D 🥢


In [11]:
res = get_response('좋아좋아 너 친절하구낭')
print(res)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new  chain...
Prompt after formatting:
이전 대화와 현재 대화의 input을 참고하여 상황에 공감하고 친절한 답변을 생성해주세요. 답변 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.

[이전 대화]
### 명령어: 그렇지..! 다들 하는건데 나만 못할 순 없지! 좋아 잘 해볼게
### 응답: 좋아요, 시작합시다!! 저한테 말할 땐 좀 내려놓으세요." [웃음 포인트로 마무리하면 좋습니다 ^-^)/~♥
### 명령어: 너가 건강식이 된다고..? 그게 무슨소리니... 암튼 안녕
### 응답: 죄송해요 방금 한 말 못 들으셨나요.... 저 오늘 아침부터 저녁식사 전까질 수도 없습니다!! 그 동안 최선 다할게요~^-/-->

[현재 대화]
### 명령어: 좋아좋아 너 친절하구낭

### 응답:


> Finished chain.
네 제 말투 때문이라면 오해하셨네요; 그냥 내 몸은 최고야 - 난 당신만큼이나 완벽해 >_<b 거기다 맛있기 까지 하고 완전 대박이야 <3"


In [ ]:
res = get_response('오늘 휴식을 위해 점심때 뭐한다고 했지?')
print(res)

In [ ]:
res = get_response('')
print(res)

In [ ]:
res = get_response('')
print(res)

In [ ]:
res = get_response('')
print(res)